In [12]:
import glob
import os
import imageio
import skimage as ski
import numpy as np
import matplotlib
matplotlib.use('gtk3agg')  # Use gtk3agg for GNOME
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from skimage.feature import SIFT, match_descriptors
from sklearn.cluster import KMeans

"""
Resources used:
    https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.SIFT
"""

'\nResources used:\n    https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.SIFT\n'

In [13]:
import glob
import os
import imageio
import skimage as ski
import numpy as np
import matplotlib
matplotlib.use('gtk3agg')  # Use gtk3agg for GNOME
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from skimage.feature import SIFT, match_descriptors
from sklearn.cluster import KMeans

In [14]:
def grey_list(paths):
    
    gray_list =[] #for path in paths:
    detector_list = []
    i = 0
    for path in paths:    
        image = imageio.v2.imread(path)
        gray_image = ski.color.rgb2gray(image) #this turns each pixel into a float 0-1
        gray_list.append(gray_image)
        sift_stuff(gray_image, detector_list)
    return detector_list

def sift_stuff(image, list):
    #descriptors are the local image gradient info, around the keypoint
    descriptor_extractor= SIFT()
    descriptor_extractor.detect_and_extract(image)
    list.append(descriptor_extractor.descriptors) # appending x amount of descriptors, each descriptor has 128 features

In [15]:

paths = sorted(glob.glob('Childrens-Books/*.jpg'))

sift_list = grey_list(paths)

In [35]:
for i, item in enumerate(sift_list):
    if item is not None and len(item) > 0:
        continue
    else:
        print("value not valid")

In [39]:

    #https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
kmeans = KMeans(n_clusters=1000)
flat_array = np.vstack(sift_list)
mean_list = kmeans.fit(flat_array) #needs a 1D array

In [41]:

    
cluster_list = []
for item in sift_list:
    if item is not None and len(item) > 0:
        val = kmeans.predict(item)
        cluster_list.append(val)
    else: 
        print(item, "not valid")

In [48]:
print(flat_array.shape)
print(len(sift_list[0]))
print(len(cluster_list[0]))

(387790, 128)
306
306


In [43]:
feature_list = []
for val in cluster_list:
    feature = kmeans.get_feature_names_out(val)
    feature_list.append(feature)

ValueError: input_features should have length equal to number of features (128), got 306

In [ ]:

cluster_list = sift_average(sift_list)